<a href="https://colab.research.google.com/github/AlvaroMartinez87/creating-a-pipeline-in-blue-ocean/blob/master/PyCuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejemplo de PyCuda**

## **Instalación**
NOTA: En Google Colab, el entorno de ejecución tiene que disponer de una GPU


In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=5d080dc94de9c812d3367cadf5c37c1e1dfb3698157d61ed22c4fa48b748a9c4
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.9-py2.py3-none-any.whl size=64666 sha256=f42fe7f92a1a0aecbb7de6eb0f23b234de1944eca9f25c2a24790fbb7013b0a0
  Stored in directory: /root/.cache/pip/wheels/11/d8/85/01a8cf50977badf2da880865124b159b112b861655a9b631d3
Successfully built pycuda pytools


Importar y inicializar PyCuda:

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

## **Transferir datos**

Creamos primero un array numpy de números aleatorios 4x4:


In [3]:
import numpy
A = numpy.random.randn(4,4)

El array `A` está compuesto por valores de doble precisión (float64). Como la mayoría de las GPU sólo soportan simple precisión, vamos a convertir los valores de `A` en float32

In [4]:
A = A.astype(numpy.float32)

 Reservamos memoria en la GPU

In [5]:
A_gpu = cuda.mem_alloc(A.nbytes)

 Tranferimos los datos del *host* al *device* (GPU)

In [6]:
cuda.memcpy_htod(A_gpu, A)

# **Ejecutado el Kernel**

La función kernel simplemente va a duplicar el valor de cada elemento del array. Tenemos que escribir el código CUDA C del kernel en el constructor de `pycuda.compiler.SourceModule`:

In [7]:
mod = SourceModule("""
  __global__ void duplicar(float *A)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    A[idx] *= 2;
  }
  """)

Si no hay errores, el código se compila y se carga en la GPU. Encontramos una referencia a nuestra función en `pycuda.driver.Function`. Y la llamamos, pasando `A_gpu` como primer argumento, y un bloque de tamaño *4x4* como segundo:

In [8]:
func_duplicar = mod.get_function("duplicar")
func_duplicar(A_gpu, block=(4,4,1))

Finalmente, transferimos los datos del *device* al *host* y mostramos el resultado junto con los valores originales:

In [9]:
A_doble = numpy.empty_like(A)
cuda.memcpy_dtoh(A_doble, A_gpu)
print(A_doble)
print()
print(A)

[[-0.27930433  3.1280973   1.3525409  -0.55110455]
 [-0.16594541  0.66767275 -0.32070175 -0.2876671 ]
 [ 1.8403378   1.3806794   0.9448152   0.7676457 ]
 [-3.9800985   0.8526725  -0.05736419  0.2632798 ]]

[[-0.13965216  1.5640486   0.6762704  -0.27555227]
 [-0.08297271  0.33383638 -0.16035087 -0.14383355]
 [ 0.9201689   0.6903397   0.4724076   0.38382286]
 [-1.9900492   0.42633626 -0.02868209  0.1316399 ]]
